# JUICE RPWI HF SID6 (PSSR2 surv): L1a QL -- 2025/3/31

# Import lib

In [ ]:
import sys
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors

# CDF and Directory setting: set by User

In [ ]:
# The latest CDF library https://spdf.gsfc.nasa.gov/pub/software/cdf/dist/latest/
from spacepy import pycdf

import os
os.environ["CDF_LIB"] = "/Applications/cdf/cdf39_1-dist/lib"

# *** Library ***
sys.path.append('./lib/')
import juice_cdf_lib as juice_cdf
import juice_math_lib as juice_math
import juice_sid6_lib as juice_sid6

# Setting and Read CDF file: set by User

In [ ]:
# *** Mode set ***
band_mode = 0                           # 0: sum    1: /Hz
unit_mode = 0                           # 0: raw    1: dBm＠ADC  2: V@HF   3:V2@HF   4:V2@RWI
f_mode = 0                              # 0: linear   1:log  in frequency
dump_mode = 1                           # 0: no-dump  1:plot dump

# *** Parameter set ***
cal = 0                                 # 0: background     1: CAL
p_raw_max = 8.5                         # background: 7.5   CAL: 10
p_raw_min = 1.0                         # background: 2.5   CAL: 5

# *** Conversion factor: cal_mode ***
# cf, p_max0, p_min0, str_unit = juice_cdf.cal_factors(unit_mode, p_raw_max, p_raw_min)
# print("conversion factor:", cf, "    MAX-min:", p_max0, p_min0, "   str_unit:", str_unit)

# *** Directory set: set by User ***
work_dir = '/Users/user/0-python/JUICE_python/ql/'                         # Plot dump folder

In [ ]:
# *** Flight data: Ver.2 ***
"""
# 20240126
data_dir = '/Users/user/Dropbox-Univ/data/data-JUICE/datasets/'
data_name_list = ['2024/01/26/JUICE_L1a_RPWI-HF-SID6_20240126T091728_V01.cdf']"
"""

In [ ]:
# *** Ver.2 ***
# 202310 -- SAMPLE
data_dir = '/Users/user/0-python/JUICE_data/test-CCSDS/sample-ASW2/cdf/old/'
# data_dir = '/Users/user/0-python/JUICE_data/test-CCSDS/sample-ASW2/cdf/old2/'
data_name_list = ['SID06_20231024-0049.ccs.cdf']
"""
# 202310 -- FS
data_dir = '/Users/user/0-python/JUICE_data/test-TMIDX/202310_FS/cdf/'    # CDF data folder
data_name_list = ['SID06_Seq06.cdf']
"""

In [ ]:
# *** Group read
"""
data_dir = '/Users/user/0-python/JUICE_data/Data-CDF/2023/05/23/'        # CDF data folder
data_name = '*SID4*.cdf'
cdf_file = data_dir + data_name
data_name_list = glob.glob(cdf_file)
num_list = len(data_name_list)
data_name_list.sort()
for i in range(num_list):
    data_name_list[i] = os.path.split(data_name_list[i])[1]
"""

# get CDF data

In [ ]:
class struct:
    pass

data = struct()
num_list = len(data_name_list)

for i in range(num_list):
    data_name = data_name_list[i]
    cdf_file = data_dir + data_name
    print(i, cdf_file)

    RPWI_FSW_version = juice_cdf._RPWI_FSW_version(cdf_file)
    cdf = pycdf.CDF(cdf_file);      data1 = juice_sid6.hf_sid6_read(cdf, RPWI_FSW_version)

    if i==0:
        data = data1
        print(data.auto_corr.shape)
    else:
        data = juice_sid6.hf_sid6_add(data, data1)
        print(data.auto_corr.shape)    

data_name = os.path.split(data_name)[1];  print("data name:", data_name)

In [ ]:
"""
cdf_file = data_dir + data_name
%ls $cdf_file

cdf = pycdf.CDF(cdf_file)
data = juice_sid6.juice_getdata_hf_sid6(cdf)
"""
data = juice_sid6.hf_sid6_shaping(data)

date1 = data.epoch[0];  date1 = date1.strftime('%Y/%m/%d %R:%S')
date2 = data.epoch[-1]; date2 = date2.strftime('%Y/%m/%d %R:%S')
str_date = date1 + "  -  " + date2;  print(str_date)

n_time = data.auto_corr.shape[0]
n_time0 = n_time-1
print("data size:", data.auto_corr.shape)

In [ ]:
# Mode 
print("==> Ch:", data.ch_selected[0], "  Num-sweep:", n_time, "   Num-AutoCorr:", data.N_step[0], "   each Length:", data.N_auto_corr[0] )

# Raw data

In [ ]:
fig = plt.figure(figsize=(14, 11))
ax1 = fig.add_subplot(3, 1, 1);  ax2 = fig.add_subplot(3, 1, 2);  ax3 = fig.add_subplot(3, 1, 3)

ax1.plot(np.ravel(data.auto_corr[:][:]), linewidth=.5, label='auto_corr')
ax2.plot(np.ravel(data.epoch[:]), '.')
ax3.plot(np.ravel(data.T_HF_FPGA),  ':r', label='T (HK-FPGA)')
ax3.plot(np.ravel(data.T_RWI_CH1),  ':b', label='T (RWI1)')
ax3.plot(np.ravel(data.T_RWI_CH2),  ':g', label='T (RWI2)')
#
ax1.set_ylabel('AutoCorr');  ax3.set_ylabel('T [degC]')
ax2.set_ylabel('Date-Time')
ax2.set_xlabel(str_date)
#
title_label = '[JUICE/RPWI HF PSSR2-Surv (SID-6)]  ' + data_name
ax1.set_title(title_label)
ax1.legend(loc='upper right', fontsize=8); ax3.legend(loc='upper right', fontsize=8); 

xlim=[-.5, len(np.ravel(data.auto_corr))     -.5];  print(xlim);  ax1.set_xlim(xlim)
xlim=[-.5, len(np.ravel(data.epoch))         -.5];  print(xlim);  ax2.set_xlim(xlim);  ax3.set_xlim(xlim)
# ylim=[10**p_min0, 10**p_max0];  ax1.set_ylim(ylim)

# Plot
fig.show
if dump_mode == 1:
    png_fname = work_dir+data_name+'_raw.png'
    fig.savefig(png_fname)

# print(data.T_HF_FPGA, data.T_RWI_CH1, data.T_RWI_CH2)

## First data

In [ ]:
# Sweep_num
n0 = 0
n1 = np.int16(data.N_auto_corr[0]/2)
n2 = data.N_auto_corr[0]-1

n_sweep = 0

fig = plt.figure(figsize=(14, 11))
ax1  = fig.add_subplot(4, 4, 1);  ax2  = fig.add_subplot(4, 4, 2);   ax3  = fig.add_subplot(4, 4, 3);   ax4 = fig.add_subplot(4, 4, 4)
ax5  = fig.add_subplot(4, 4, 5);  ax6  = fig.add_subplot(4, 4, 6);   ax7  = fig.add_subplot(4, 4, 7);   ax8 = fig.add_subplot(4, 4, 8)
ax9  = fig.add_subplot(4, 4, 9);  ax10 = fig.add_subplot(4, 4, 10);  ax11 = fig.add_subplot(4, 4, 11); ax12 = fig.add_subplot(4, 4, 12)
ax13 = fig.add_subplot(4, 4, 13); ax14 = fig.add_subplot(4, 4, 14);  ax15 = fig.add_subplot(4, 4, 15); ax16 = fig.add_subplot(4, 4, 16)

ax1.plot(data.time, data.auto_corr[n_sweep][0][:], '-r', linewidth=.5, label='1 MHz')
ax1.plot(data.time, data.auto_corr[n_sweep][1][:], '-g', linewidth=.5, label='2 MHz')
ax1.plot(data.time, data.auto_corr[n_sweep][2][:], '-b', linewidth=.5, label='3 MHz')
ax2.plot(data.time, data.auto_corr[n_sweep][3][:], '-r', linewidth=.5, label='4 MHz')
ax2.plot(data.time, data.auto_corr[n_sweep][4][:], '-g', linewidth=.5, label='5 MHz')
ax2.plot(data.time, data.auto_corr[n_sweep][5][:], '-b', linewidth=.5, label='6 MHz')
ax3.plot(data.time, data.auto_corr[n_sweep][6][:], '-r', linewidth=.5, label='7 MHz')
ax3.plot(data.time, data.auto_corr[n_sweep][7][:], '-g', linewidth=.5, label='8 MHz')
ax3.plot(data.time, data.auto_corr[n_sweep][8][:], '-b', linewidth=.5, label='9 MHz')
ax4.plot(data.time, data.auto_corr[n_sweep][9][:], '-r', linewidth=.5, label='10 MHz')
ax4.plot(data.time, data.auto_corr[n_sweep][10][:], '-g', linewidth=.5, label='11 MHz')
ax4.plot(data.time, data.auto_corr[n_sweep][11][:], '-b', linewidth=.5, label='12 MHz')
ax5.plot(data.time, data.auto_corr[n_sweep][12][:], '-r', linewidth=.5, label='13 MHz')
ax5.plot(data.time, data.auto_corr[n_sweep][13][:], '-g', linewidth=.5, label='14 MHz')
ax5.plot(data.time, data.auto_corr[n_sweep][14][:], '-b', linewidth=.5, label='15 MHz')
ax6.plot(data.time, data.auto_corr[n_sweep][15][:], '-r', linewidth=.5, label='16 MHz')
ax6.plot(data.time, data.auto_corr[n_sweep][16][:], '-g', linewidth=.5, label='17 MHz')
ax6.plot(data.time, data.auto_corr[n_sweep][17][:], '-b', linewidth=.5, label='18 MHz')
ax7.plot(data.time, data.auto_corr[n_sweep][18][:], '-r', linewidth=.5, label='19 MHz')
ax7.plot(data.time, data.auto_corr[n_sweep][19][:], '-g', linewidth=.5, label='20 MHz')
ax7.plot(data.time, data.auto_corr[n_sweep][20][:], '-b', linewidth=.5, label='21 MHz')
ax8.plot(data.time, data.auto_corr[n_sweep][21][:], '-r', linewidth=.5, label='22 MHz')
ax8.plot(data.time, data.auto_corr[n_sweep][22][:], '-g', linewidth=.5, label='23 MHz')
ax8.plot(data.time, data.auto_corr[n_sweep][23][:], '-b', linewidth=.5, label='24 MHz')
ax9.plot(data.time, data.auto_corr[n_sweep][24][:], '-r', linewidth=.5, label='25 MHz')
ax9.plot(data.time, data.auto_corr[n_sweep][25][:], '-g', linewidth=.5, label='26 MHz')
ax9.plot(data.time, data.auto_corr[n_sweep][26][:], '-b', linewidth=.5, label='27 MHz')
ax10.plot(data.time, data.auto_corr[n_sweep][27][:], '-r', linewidth=.5, label='28 MHz')
ax10.plot(data.time, data.auto_corr[n_sweep][28][:], '-g', linewidth=.5, label='29 MHz')
ax10.plot(data.time, data.auto_corr[n_sweep][29][:], '-b', linewidth=.5, label='30 MHz')
ax11.plot(data.time, data.auto_corr[n_sweep][30][:], '-r', linewidth=.5, label='31 MHz')
ax11.plot(data.time, data.auto_corr[n_sweep][31][:], '-g', linewidth=.5, label='32 MHz')
ax11.plot(data.time, data.auto_corr[n_sweep][32][:], '-b', linewidth=.5, label='33 MHz')
ax12.plot(data.time, data.auto_corr[n_sweep][33][:], '-r', linewidth=.5, label='34 MHz')
ax12.plot(data.time, data.auto_corr[n_sweep][34][:], '-g', linewidth=.5, label='35 MHz')
ax12.plot(data.time, data.auto_corr[n_sweep][35][:], '-b', linewidth=.5, label='36 MHz')
ax13.plot(data.time, data.auto_corr[n_sweep][36][:], '-r', linewidth=.5, label='37 MHz')
ax13.plot(data.time, data.auto_corr[n_sweep][37][:], '-g', linewidth=.5, label='38 MHz')
ax13.plot(data.time, data.auto_corr[n_sweep][38][:], '-b', linewidth=.5, label='39 MHz')
ax14.plot(data.time, data.auto_corr[n_sweep][39][:], '-r', linewidth=.5, label='40 MHz')
ax14.plot(data.time, data.auto_corr[n_sweep][40][:], '-g', linewidth=.5, label='41 MHz')
ax14.plot(data.time, data.auto_corr[n_sweep][41][:], '-b', linewidth=.5, label='42 MHz')
ax15.plot(data.time, data.auto_corr[n_sweep][42][:], '-r', linewidth=.5, label='43 MHz')
ax15.plot(data.time, data.auto_corr[n_sweep][43][:], '-g', linewidth=.5, label='44 MHz')
ax15.plot(data.time, data.auto_corr[n_sweep][44][:], '-b', linewidth=.5, label='45 MHz')
# ax16.plot(data.time, data.auto_corr[n_sweep][45][:], '-r', linewidth=.5)
# ax16.plot(data.time, data.auto_corr[n_sweep][46][:], '-g', linewidth=.5)
# ax16.plot(data.time, data.auto_corr[n_sweep][47][:], '-gb', linewidth=.5)
auto_corr = data.auto_corr[n_sweep]
auto_corr = np.transpose(auto_corr)
print(auto_corr.shape)

auto_corr1 = auto_corr[n0][0:45]
auto_corr2 = auto_corr[n1][0:45]
auto_corr3 = auto_corr[n2][0:45]
ax16.plot(data.freq, auto_corr1, '-r', linewidth=.5, label='first')
ax16.plot(data.freq, auto_corr2, '-g', linewidth=.5, label='mid')
ax16.plot(data.freq, auto_corr3, '-b', linewidth=.5, label='last')
#
date1 = data.epoch[n_sweep];  date1 = date1.strftime('First: %Y/%m/%d %R:%S')
title_date = "[" + data_name + "]  " + date1;  ax2.set_title(title_date)
ax1.legend(loc='upper right', fontsize=8);  ax2.legend(loc='upper right', fontsize=8);   ax3.legend(loc='upper right', fontsize=8)
ax4.legend(loc='upper right', fontsize=8);  ax5.legend(loc='upper right', fontsize=8);   ax6.legend(loc='upper right', fontsize=8)
ax7.legend(loc='upper right', fontsize=8);  ax8.legend(loc='upper right', fontsize=8);   ax9.legend(loc='upper right', fontsize=8)
ax10.legend(loc='upper right', fontsize=8); ax11.legend(loc='upper right', fontsize=8);  ax12.legend(loc='upper right', fontsize=8)
ax10.legend(loc='upper right', fontsize=8); ax11.legend(loc='upper right', fontsize=8);  ax12.legend(loc='upper right', fontsize=8)
ax13.legend(loc='upper right', fontsize=8); ax14.legend(loc='upper right', fontsize=8);  ax15.legend(loc='upper right', fontsize=8)
ax16.legend(loc='upper right', fontsize=8)
ax13.set_xlabel("Time [sec]");              ax14.set_xlabel("Time [sec]");               ax15.set_xlabel("Time [sec]")
# ax16.set_xlabel("(46 - 48)  Time [sec]")
ax16.set_xlabel("Frequency [kHz]")
#
ax1.legend(loc='upper right', fontsize=8)

fig.subplots_adjust(hspace=0)
fig.show
if dump_mode == 1:
    png_fname = work_dir+data_name+'_1st.png'
    fig.savefig(png_fname)

## Mid data

In [ ]:
n_sweep = np.int16(n_time0/2)

fig = plt.figure(figsize=(14, 11))
ax1  = fig.add_subplot(4, 4, 1);  ax2  = fig.add_subplot(4, 4, 2);   ax3  = fig.add_subplot(4, 4, 3);   ax4 = fig.add_subplot(4, 4, 4)
ax5  = fig.add_subplot(4, 4, 5);  ax6  = fig.add_subplot(4, 4, 6);   ax7  = fig.add_subplot(4, 4, 7);   ax8 = fig.add_subplot(4, 4, 8)
ax9  = fig.add_subplot(4, 4, 9);  ax10 = fig.add_subplot(4, 4, 10);  ax11 = fig.add_subplot(4, 4, 11); ax12 = fig.add_subplot(4, 4, 12)
ax13 = fig.add_subplot(4, 4, 13); ax14 = fig.add_subplot(4, 4, 14);  ax15 = fig.add_subplot(4, 4, 15); ax16 = fig.add_subplot(4, 4, 16)

ax1.plot(data.time, data.auto_corr[n_sweep][0][:], '-r', linewidth=.5, label='1 MHz')
ax1.plot(data.time, data.auto_corr[n_sweep][1][:], '-g', linewidth=.5, label='2 MHz')
ax1.plot(data.time, data.auto_corr[n_sweep][2][:], '-b', linewidth=.5, label='3 MHz')
ax2.plot(data.time, data.auto_corr[n_sweep][3][:], '-r', linewidth=.5, label='4 MHz')
ax2.plot(data.time, data.auto_corr[n_sweep][4][:], '-g', linewidth=.5, label='5 MHz')
ax2.plot(data.time, data.auto_corr[n_sweep][5][:], '-b', linewidth=.5, label='6 MHz')
ax3.plot(data.time, data.auto_corr[n_sweep][6][:], '-r', linewidth=.5, label='7 MHz')
ax3.plot(data.time, data.auto_corr[n_sweep][7][:], '-g', linewidth=.5, label='8 MHz')
ax3.plot(data.time, data.auto_corr[n_sweep][8][:], '-b', linewidth=.5, label='9 MHz')
ax4.plot(data.time, data.auto_corr[n_sweep][9][:], '-r', linewidth=.5, label='10 MHz')
ax4.plot(data.time, data.auto_corr[n_sweep][10][:], '-g', linewidth=.5, label='11 MHz')
ax4.plot(data.time, data.auto_corr[n_sweep][11][:], '-b', linewidth=.5, label='12 MHz')
ax5.plot(data.time, data.auto_corr[n_sweep][12][:], '-r', linewidth=.5, label='13 MHz')
ax5.plot(data.time, data.auto_corr[n_sweep][13][:], '-g', linewidth=.5, label='14 MHz')
ax5.plot(data.time, data.auto_corr[n_sweep][14][:], '-b', linewidth=.5, label='15 MHz')
ax6.plot(data.time, data.auto_corr[n_sweep][15][:], '-r', linewidth=.5, label='16 MHz')
ax6.plot(data.time, data.auto_corr[n_sweep][16][:], '-g', linewidth=.5, label='17 MHz')
ax6.plot(data.time, data.auto_corr[n_sweep][17][:], '-b', linewidth=.5, label='18 MHz')
ax7.plot(data.time, data.auto_corr[n_sweep][18][:], '-r', linewidth=.5, label='19 MHz')
ax7.plot(data.time, data.auto_corr[n_sweep][19][:], '-g', linewidth=.5, label='20 MHz')
ax7.plot(data.time, data.auto_corr[n_sweep][20][:], '-b', linewidth=.5, label='21 MHz')
ax8.plot(data.time, data.auto_corr[n_sweep][21][:], '-r', linewidth=.5, label='22 MHz')
ax8.plot(data.time, data.auto_corr[n_sweep][22][:], '-g', linewidth=.5, label='23 MHz')
ax8.plot(data.time, data.auto_corr[n_sweep][23][:], '-b', linewidth=.5, label='24 MHz')
ax9.plot(data.time, data.auto_corr[n_sweep][24][:], '-r', linewidth=.5, label='25 MHz')
ax9.plot(data.time, data.auto_corr[n_sweep][25][:], '-g', linewidth=.5, label='26 MHz')
ax9.plot(data.time, data.auto_corr[n_sweep][26][:], '-b', linewidth=.5, label='27 MHz')
ax10.plot(data.time, data.auto_corr[n_sweep][27][:], '-r', linewidth=.5, label='28 MHz')
ax10.plot(data.time, data.auto_corr[n_sweep][28][:], '-g', linewidth=.5, label='29 MHz')
ax10.plot(data.time, data.auto_corr[n_sweep][29][:], '-b', linewidth=.5, label='30 MHz')
ax11.plot(data.time, data.auto_corr[n_sweep][30][:], '-r', linewidth=.5, label='31 MHz')
ax11.plot(data.time, data.auto_corr[n_sweep][31][:], '-g', linewidth=.5, label='32 MHz')
ax11.plot(data.time, data.auto_corr[n_sweep][32][:], '-b', linewidth=.5, label='33 MHz')
ax12.plot(data.time, data.auto_corr[n_sweep][33][:], '-r', linewidth=.5, label='34 MHz')
ax12.plot(data.time, data.auto_corr[n_sweep][34][:], '-g', linewidth=.5, label='35 MHz')
ax12.plot(data.time, data.auto_corr[n_sweep][35][:], '-b', linewidth=.5, label='36 MHz')
ax13.plot(data.time, data.auto_corr[n_sweep][36][:], '-r', linewidth=.5, label='37 MHz')
ax13.plot(data.time, data.auto_corr[n_sweep][37][:], '-g', linewidth=.5, label='38 MHz')
ax13.plot(data.time, data.auto_corr[n_sweep][38][:], '-b', linewidth=.5, label='39 MHz')
ax14.plot(data.time, data.auto_corr[n_sweep][39][:], '-r', linewidth=.5, label='40 MHz')
ax14.plot(data.time, data.auto_corr[n_sweep][40][:], '-g', linewidth=.5, label='41 MHz')
ax14.plot(data.time, data.auto_corr[n_sweep][41][:], '-b', linewidth=.5, label='42 MHz')
ax15.plot(data.time, data.auto_corr[n_sweep][42][:], '-r', linewidth=.5, label='43 MHz')
ax15.plot(data.time, data.auto_corr[n_sweep][43][:], '-g', linewidth=.5, label='44 MHz')
ax15.plot(data.time, data.auto_corr[n_sweep][44][:], '-b', linewidth=.5, label='45 MHz')
# ax16.plot(data.time, data.auto_corr[n_sweep][45][:], '-r', linewidth=.5)
# ax16.plot(data.time, data.auto_corr[n_sweep][46][:], '-g', linewidth=.5)
# ax16.plot(data.time, data.auto_corr[n_sweep][47][:], '-gb', linewidth=.5)
auto_corr = data.auto_corr[n_sweep]
auto_corr = np.transpose(auto_corr)
print(auto_corr.shape)

auto_corr1 = auto_corr[n0][0:45]
auto_corr2 = auto_corr[n1][0:45]
auto_corr3 = auto_corr[n2][0:45]
ax16.plot(data.freq, auto_corr1, '-r', linewidth=.5, label='first')
ax16.plot(data.freq, auto_corr2, '-g', linewidth=.5, label='mid')
ax16.plot(data.freq, auto_corr3, '-b', linewidth=.5, label='last')
#
date1 = data.epoch[n_sweep];  date1 = date1.strftime('Mid: %Y/%m/%d %R:%S')
title_date = "[" + data_name + "]  " + date1;  ax2.set_title(title_date)
ax1.legend(loc='upper right', fontsize=8);  ax2.legend(loc='upper right', fontsize=8);   ax3.legend(loc='upper right', fontsize=8)
ax4.legend(loc='upper right', fontsize=8);  ax5.legend(loc='upper right', fontsize=8);   ax6.legend(loc='upper right', fontsize=8)
ax7.legend(loc='upper right', fontsize=8);  ax8.legend(loc='upper right', fontsize=8);   ax9.legend(loc='upper right', fontsize=8)
ax10.legend(loc='upper right', fontsize=8); ax11.legend(loc='upper right', fontsize=8);  ax12.legend(loc='upper right', fontsize=8)
ax10.legend(loc='upper right', fontsize=8); ax11.legend(loc='upper right', fontsize=8);  ax12.legend(loc='upper right', fontsize=8)
ax13.legend(loc='upper right', fontsize=8); ax14.legend(loc='upper right', fontsize=8);  ax15.legend(loc='upper right', fontsize=8)
ax16.legend(loc='upper right', fontsize=8)
ax13.set_xlabel("Time [sec]");              ax14.set_xlabel("Time [sec]");               ax15.set_xlabel("Time [sec]")
# ax16.set_xlabel("(46 - 48)  Time [sec]")
ax16.set_xlabel("Frequency [kHz]")
#
ax1.legend(loc='upper right', fontsize=8)

fig.subplots_adjust(hspace=0)
fig.show
if dump_mode == 1:
    png_fname = work_dir+data_name+'_mid.png'
    fig.savefig(png_fname)

## Last data

In [ ]:
n_sweep = n_time0

fig = plt.figure(figsize=(14, 11))
ax1  = fig.add_subplot(4, 4, 1);  ax2  = fig.add_subplot(4, 4, 2);   ax3  = fig.add_subplot(4, 4, 3);   ax4 = fig.add_subplot(4, 4, 4)
ax5  = fig.add_subplot(4, 4, 5);  ax6  = fig.add_subplot(4, 4, 6);   ax7  = fig.add_subplot(4, 4, 7);   ax8 = fig.add_subplot(4, 4, 8)
ax9  = fig.add_subplot(4, 4, 9);  ax10 = fig.add_subplot(4, 4, 10);  ax11 = fig.add_subplot(4, 4, 11); ax12 = fig.add_subplot(4, 4, 12)
ax13 = fig.add_subplot(4, 4, 13); ax14 = fig.add_subplot(4, 4, 14);  ax15 = fig.add_subplot(4, 4, 15); ax16 = fig.add_subplot(4, 4, 16)

ax1.plot(data.time, data.auto_corr[n_sweep][0][:], '-r', linewidth=.5, label='1 MHz')
ax1.plot(data.time, data.auto_corr[n_sweep][1][:], '-g', linewidth=.5, label='2 MHz')
ax1.plot(data.time, data.auto_corr[n_sweep][2][:], '-b', linewidth=.5, label='3 MHz')
ax2.plot(data.time, data.auto_corr[n_sweep][3][:], '-r', linewidth=.5, label='4 MHz')
ax2.plot(data.time, data.auto_corr[n_sweep][4][:], '-g', linewidth=.5, label='5 MHz')
ax2.plot(data.time, data.auto_corr[n_sweep][5][:], '-b', linewidth=.5, label='6 MHz')
ax3.plot(data.time, data.auto_corr[n_sweep][6][:], '-r', linewidth=.5, label='7 MHz')
ax3.plot(data.time, data.auto_corr[n_sweep][7][:], '-g', linewidth=.5, label='8 MHz')
ax3.plot(data.time, data.auto_corr[n_sweep][8][:], '-b', linewidth=.5, label='9 MHz')
ax4.plot(data.time, data.auto_corr[n_sweep][9][:], '-r', linewidth=.5, label='10 MHz')
ax4.plot(data.time, data.auto_corr[n_sweep][10][:], '-g', linewidth=.5, label='11 MHz')
ax4.plot(data.time, data.auto_corr[n_sweep][11][:], '-b', linewidth=.5, label='12 MHz')
ax5.plot(data.time, data.auto_corr[n_sweep][12][:], '-r', linewidth=.5, label='13 MHz')
ax5.plot(data.time, data.auto_corr[n_sweep][13][:], '-g', linewidth=.5, label='14 MHz')
ax5.plot(data.time, data.auto_corr[n_sweep][14][:], '-b', linewidth=.5, label='15 MHz')
ax6.plot(data.time, data.auto_corr[n_sweep][15][:], '-r', linewidth=.5, label='16 MHz')
ax6.plot(data.time, data.auto_corr[n_sweep][16][:], '-g', linewidth=.5, label='17 MHz')
ax6.plot(data.time, data.auto_corr[n_sweep][17][:], '-b', linewidth=.5, label='18 MHz')
ax7.plot(data.time, data.auto_corr[n_sweep][18][:], '-r', linewidth=.5, label='19 MHz')
ax7.plot(data.time, data.auto_corr[n_sweep][19][:], '-g', linewidth=.5, label='20 MHz')
ax7.plot(data.time, data.auto_corr[n_sweep][20][:], '-b', linewidth=.5, label='21 MHz')
ax8.plot(data.time, data.auto_corr[n_sweep][21][:], '-r', linewidth=.5, label='22 MHz')
ax8.plot(data.time, data.auto_corr[n_sweep][22][:], '-g', linewidth=.5, label='23 MHz')
ax8.plot(data.time, data.auto_corr[n_sweep][23][:], '-b', linewidth=.5, label='24 MHz')
ax9.plot(data.time, data.auto_corr[n_sweep][24][:], '-r', linewidth=.5, label='25 MHz')
ax9.plot(data.time, data.auto_corr[n_sweep][25][:], '-g', linewidth=.5, label='26 MHz')
ax9.plot(data.time, data.auto_corr[n_sweep][26][:], '-b', linewidth=.5, label='27 MHz')
ax10.plot(data.time, data.auto_corr[n_sweep][27][:], '-r', linewidth=.5, label='28 MHz')
ax10.plot(data.time, data.auto_corr[n_sweep][28][:], '-g', linewidth=.5, label='29 MHz')
ax10.plot(data.time, data.auto_corr[n_sweep][29][:], '-b', linewidth=.5, label='30 MHz')
ax11.plot(data.time, data.auto_corr[n_sweep][30][:], '-r', linewidth=.5, label='31 MHz')
ax11.plot(data.time, data.auto_corr[n_sweep][31][:], '-g', linewidth=.5, label='32 MHz')
ax11.plot(data.time, data.auto_corr[n_sweep][32][:], '-b', linewidth=.5, label='33 MHz')
ax12.plot(data.time, data.auto_corr[n_sweep][33][:], '-r', linewidth=.5, label='34 MHz')
ax12.plot(data.time, data.auto_corr[n_sweep][34][:], '-g', linewidth=.5, label='35 MHz')
ax12.plot(data.time, data.auto_corr[n_sweep][35][:], '-b', linewidth=.5, label='36 MHz')
ax13.plot(data.time, data.auto_corr[n_sweep][36][:], '-r', linewidth=.5, label='37 MHz')
ax13.plot(data.time, data.auto_corr[n_sweep][37][:], '-g', linewidth=.5, label='38 MHz')
ax13.plot(data.time, data.auto_corr[n_sweep][38][:], '-b', linewidth=.5, label='39 MHz')
ax14.plot(data.time, data.auto_corr[n_sweep][39][:], '-r', linewidth=.5, label='40 MHz')
ax14.plot(data.time, data.auto_corr[n_sweep][40][:], '-g', linewidth=.5, label='41 MHz')
ax14.plot(data.time, data.auto_corr[n_sweep][41][:], '-b', linewidth=.5, label='42 MHz')
ax15.plot(data.time, data.auto_corr[n_sweep][42][:], '-r', linewidth=.5, label='43 MHz')
ax15.plot(data.time, data.auto_corr[n_sweep][43][:], '-g', linewidth=.5, label='44 MHz')
ax15.plot(data.time, data.auto_corr[n_sweep][44][:], '-b', linewidth=.5, label='45 MHz')
# ax16.plot(data.time, data.auto_corr[n_sweep][45][:], '-r', linewidth=.5)
# ax16.plot(data.time, data.auto_corr[n_sweep][46][:], '-g', linewidth=.5)
# ax16.plot(data.time, data.auto_corr[n_sweep][47][:], '-gb', linewidth=.5)
auto_corr = data.auto_corr[n_sweep]
auto_corr = np.transpose(auto_corr)
print(auto_corr.shape)

auto_corr1 = auto_corr[n0][0:45]
auto_corr2 = auto_corr[n1][0:45]
auto_corr3 = auto_corr[n2][0:45]
ax16.plot(data.freq, auto_corr1, '-r', linewidth=.5, label='first')
ax16.plot(data.freq, auto_corr2, '-g', linewidth=.5, label='mid')
ax16.plot(data.freq, auto_corr3, '-b', linewidth=.5, label='last')
#
date1 = data.epoch[n_sweep];  date1 = date1.strftime('Last: %Y/%m/%d %R:%S')
title_date = "[" + data_name + "]  " + date1;  ax2.set_title(title_date)
ax1.legend(loc='upper right', fontsize=8);  ax2.legend(loc='upper right', fontsize=8);   ax3.legend(loc='upper right', fontsize=8)
ax4.legend(loc='upper right', fontsize=8);  ax5.legend(loc='upper right', fontsize=8);   ax6.legend(loc='upper right', fontsize=8)
ax7.legend(loc='upper right', fontsize=8);  ax8.legend(loc='upper right', fontsize=8);   ax9.legend(loc='upper right', fontsize=8)
ax10.legend(loc='upper right', fontsize=8); ax11.legend(loc='upper right', fontsize=8);  ax12.legend(loc='upper right', fontsize=8)
ax10.legend(loc='upper right', fontsize=8); ax11.legend(loc='upper right', fontsize=8);  ax12.legend(loc='upper right', fontsize=8)
ax13.legend(loc='upper right', fontsize=8); ax14.legend(loc='upper right', fontsize=8);  ax15.legend(loc='upper right', fontsize=8)
ax16.legend(loc='upper right', fontsize=8)
ax13.set_xlabel("Time [sec]");              ax14.set_xlabel("Time [sec]");               ax15.set_xlabel("Time [sec]")
# ax16.set_xlabel("(46 - 48)  Time [sec]")
ax16.set_xlabel("Frequency [kHz]")
#
ax1.legend(loc='upper right', fontsize=8)

fig.subplots_adjust(hspace=0)
fig.show
if dump_mode == 1:
    png_fname = work_dir+data_name+'_last.png'
    fig.savefig(png_fname)